# Evolutionary Strategies for FPGA

We want to generate a population of individuals (in our case it will be $1,800$-bit strings). These strings will then be used to set a configuration of the FPGA. Using evolutionary algorithms to update these $1,800$-bit strings, we hope to:
1. Approximate basic functions (1 input, 1 output), e.g. $sin(x)$, $cos(x)$
2. More complicated functions (multiple input, 1 output), e.g. [Rastrigin function](https://en.wikipedia.org/wiki/Rastrigin_function), simple DNN applications
3. Applications, e.g. image classification MNIST

## Potential Pipeline

Arbitrary Deterministic (in Evolution) Method:
1. Initialize generation $g$ of 100 random $1,800$-bit strings from $Uniform$ distribution
2. Draw random sample of $x$ values for $y = sin(x)$
3. Convert those $x$ values into $kHz$
4. Configure FPGA using the 100 random $1,800$-bit strings
5. For each configuration of FPGA: 
    1. Send the $x$ values in $kHz$ to FPGA 
    2. Get response in volts
    3. Convert response in volts $V$ to $y$ range
    4. Use MSE / MAE? to calculate how badly the FPGA configuration was
    5. Keep individuals ($1,800$-bit strings) in generation $g+1$ that are either:
        1. Lowest 10\% of MSE / MAE
        2. Lowest 50\% of MSE / MAE $\cap$ Top 10\% with largest hamming distance from generation $g - 1$'s population (Novelty aspect)

Probabilistic Method:
1. Initialize generation $g$ of 100 random $1,800$-bit strings from $Binomial(n=1,800, p=)$... (*NEED TO FIND OUT WHAT IS THE PROBABILITY DISTRIBUTION OF DRAWING RANDOM N-BIT STRINGS*)
2. Draw random sample of $x$ values for $y = sin(x)$
3. Convert those $x$ values into $kHz$
4. Configure FPGA using the 100 random $1,800$-bit strings
5. For each configuration of FPGA: 
    1. Send the $x$ values in $kHz$ to FPGA 
    2. Get response in volts
    3. Convert response in volts $V$ to $y$ range
    4. Use MSE / MAE? to calculate how badly the FPGA configuration was
    5. Keep individuals ($1,800$-bit strings) in generation $g+1$ that are either:
        1. Sample 
        2. Use the hamming-distance-based $k$-NN Novelty scores to  increase the probabilities that novel $1,800$-bit strings will be sampled

### What we still need
1. A mapping of interpretable numerical input $x$ to frequency domain $kHz$
2. A mapping of volts $V$ output to interpretable numerical $y = sin(x)$
3. Conversion of traditional loss functions like $l_1, l_2$ loss to *hamming distance*-related loss so that we can properly evaluate the fitness of the bit-strings we use to configure the FPGA to perform evolutionary strategies
4. Calculate $k$ Nearest Neighbors using *hamming distance* between generation $(g + 1)$'s bit-strings and previous generation $g$'s bit-strings for novelty search implementation

### Table of Contents
1. [Simple Genetic Algo](#sga)
2. [Novelty Search - Evolutionary Strategy](#ns-es)

In [1]:
# Library
import random

# Number of bits for strings
N = 1800

print('Random 1,800 bit string: \n\n{}'.format(bin(random.getrandbits(N))[2:]))

Random 1,800 bit string: 

1110010101011000000000010000101000101001101001000001110011101101110000010011011110010111110110000101010110111010110000110000000000100001001011111100111001011010101001001101100010000111010111000111100100111110110000110100011110110101100101101100010111111101111010110011000011010101010111110101001101000010011101101111111110111001111000010111010101101000110111001000100001110000011001111111101111101111101011011111110011111111100010010000110101001110010011010000111110001010111111001001001111011101110010110100001011010111101111000011010010010010110111010110001110100100001011010010100111010011010100011110001011010111001010001110001011000110110001011010000111111010100001111000010000011010010101001001101000000111010111101001111110001000001100101000001101001010100101111010000101110011110001010011000000101110111011001111110111111010100010000001110011111100100011101100011010010010010011011100001011011110110010000000000111101110100111101000101101110000011001110001111001100

---
## Simple Genetic Algorithm

1. Import the module

In [2]:
from pyeasyga import pyeasyga

2. Setup your data

In [3]:
# TODO: Change this for the FPGA to pass to fitness function
data = [('pear', 50), ('apple', 35), ('banana', 40)]

3. Initialise the GeneticAlgorithm class with the required data parameter, and all or some of the optional parameters

In [4]:
ga = pyeasyga.GeneticAlgorithm(
    data,
    population_size=10,
    generations=20,
    crossover_probability=0.8,
    mutation_probability=0.05,
    elitism=True,
    maximise_fitness=True
)

4. Optionally, define a function to create a representation of a candidate solution (an individual in GA speak). This function should take in the data defined in step 1. as a parameter.

In [5]:
# TODO: Change this for 1800 bit string
def create_individual(data):
    return [random.randint(0, 1) for _ in range(len(data))]

5. Set the Genetic Algorithm’s create_individual attribute to your defined function

In [6]:
ga.create_individual = create_individual

6. Optionally, define and set functions for the Genetic Algorithm’s genetic operators (i.e. crossover, mutate, selection)

In [7]:
# For the crossover function, supply two individuals (i.e. candidate
# solution representations) as parameters,
def single_point_crossover(parent_1, parent_2):
    crossover_index = random.randrange(1, len(parent_1))
    child_1 = parent_1[:crossover_index] + parent_2[crossover_index:]
    child_2 = parent_2[:crossover_index] + parent_1[crossover_index:]
    return child_1, child_2

# and set the Genetic Algorithm's ``crossover_function`` attribute to
# your defined function
ga.crossover_function = single_point_crossover

In [8]:
# For the mutate function, supply one individual (i.e. a candidate
# solution representation) as a parameter,
def mutate(individual):
    mutate_index = random.randrange(len(individual))
    if individual[mutate_index] == 0:
        individual[mutate_index] = 1
    else:
        individual[mutate_index] = 0

# and set the Genetic Algorithm's ``mutate_function`` attribute to
# your defined function
ga.mutate_function = mutate

In [9]:
# For the selection function, supply a ``population`` parameter
def selection(population):
    return random.choice(population)

# and set the Genetic Algorithm's ``selection_function`` attribute to
# your defined function
ga.selection_function = selection

7. Define a fitness function for the Genetic Algorithm. The function should take two parameters: a candidate solution representation (an individual in GA speak), and the data that is used to help determine the individual’s fitness

In [11]:
# TODO: Adjust this to measure fitness for FPGA
#       For example, to approximate sine function
def fitness(individual, data):
    fitness = 0
    if individual.count(1) == 2:
        for (selected, (fruit, profit)) in zip(individual, data):
            if selected:
                fitness += profit
    return fitness

8. Set the Genetic Algorithm’s fitness_function attribute to your defined fitness function

In [12]:
ga.fitness_function = fitness

9. Run the Genetic Algorithm

In [13]:
ga.run()

10. Print the best solution:

In [14]:
print(ga.best_individual())

(90, [1, 0, 1])


11. You can also examine all the individuals in the last generation:

In [15]:
for individual in ga.last_generation():
    print(individual)

(90, [1, 0, 1])
(85, [1, 1, 0])
(85, [1, 1, 0])
(0, [1, 1, 1])
(0, [1, 1, 1])
(0, [0, 1, 0])
(0, [1, 1, 1])
(0, [1, 1, 1])
(0, [1, 1, 1])
(0, [1, 1, 1])


---
## Covariance-Matrix Adaptation

